# Travelling Salesman Problem

In [9]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

### Manage the adjacency matrix

In [10]:
def read_matrix_file(file):
    with open(file) as matrix_file:
        matrix = [list(map(int, line.split())) for line in matrix_file]

    return matrix

def show_graph(matrix, draw_edges=False):
    G = nx.from_numpy_matrix(np.array(matrix))
    pos = nx.shell_layout(G)
    nx.draw(G, pos)

    if draw_edges:
        nx.draw_networkx_edge_labels(G, pos, label_pos=0.3)

    plt.show()

def path_to_matrix(path, matrix):
    # Creates an adjacency matrix representing the path
    nodes = range(len(matrix))
    path_matrix = np.zeros_like(matrix)

    for index in nodes:
        line = path[index]
        column = path[index + 1]

        edge_weight = matrix[line][column]
        path_matrix[line][column] = edge_weight
    
    return path_matrix

def calculate_path_cost(matrix, path):
    tsp_cost = 0
    nodes = range(len(matrix))

    for index in nodes:
        line = path[index]
        column = path[index + 1]

        edge_weight = matrix[line][column]

        tsp_cost += edge_weight

    return tsp_cost

### Brute Force

In [ ]:
# Calculate TSP cost
def brute_force_tsp(matrix, path=[0], best_cost=float("inf"), best_path=None):
    # Recursion base
    if len(path) == len(matrix):
        # Path ends on the initial node
        path.append(0)
        final_cost = calculate_path_cost(matrix, path)

        if final_cost < best_cost:
            best_path = path.copy()
            best_cost = final_cost

        path.pop()

        return path, best_cost, best_path

    # Recursive step
    for node in range(len(matrix)):
        if node in path:
            continue

        path.append(node)

        node_path, best_cost, best_path = brute_force_tsp(
            matrix, path, best_cost, best_path)

        path.pop()

    return node_path, best_cost, best_path

### Aprroximation algorithm

In [ ]:
def approximate_tsp(matrix, initial_node=0):
    # Convert adjacency matrix to MST
    MST = minimum_spanning_tree(matrix)
    MST = MST.toarray().astype(int)

    # Set initial parameters
    nodes = range(len(MST))

    path = list()
    path.append(initial_node)

    current_node = initial_node
    previous_node = -1

    # Creates a path until all nodes are connected
    while len(set(path)) != len(nodes):
        for connected_node in nodes:
            # If there's no edge, continue
            if MST[current_node, connected_node] == 0 and MST[connected_node, current_node] == 0:
                continue

            elif connected_node in path:
                continue
            
            else:
                path.append(connected_node)
                current_node = connected_node
                # Reset previous node
                previous_node = -1
                break
        else:
            # If it did not found an edge, go back to previous node
            current_node = path[previous_node]
            previous_node = previous_node - 1
            
    # Path ends on the initial node
    path.append(initial_node)

    # Calculate the TSP cost based on the path
    tsp_cost = calculate_path_cost(matrix, path)
    
    return tsp_cost, path